In [1]:
using CuArrays
using AutoGrad
using Deeplearning
using Statistics
using StatsBase
using Plots
import CSV

┌ Info: Recompiling stale cache file /Users/aliceylan/.julia/compiled/v1.1/Deeplearning/gtjFf.ji for Deeplearning [0d5bc7a0-39dd-11e9-2aab-59c8d4b467de]
└ @ Base loading.jl:1184
┌ Warning: Package Deeplearning does not have CuArrays in its dependencies:
│ - If you have Deeplearning checked out for development and have
│   added CuArrays as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with Deeplearning
└ Loading CuArrays into Deeplearning from project dependency, future warnings for Deeplearning are suppressed.


In [2]:
batch = 100
learning_rate = 10e-2

0.1

In [3]:
train = CSV.read("datasets/digit-recognizer/train.csv")
xtrain = []
ytrain = []
for i in 1:size(train)[1]
    max = maximum(train[i,2:end])+0.1
    normalized = reshape(train[i, 2:end]./(max), (28,28,1,1))
    push!(xtrain, normalized)
    push!(ytrain, train[i,1]+1)
end
println("Ready...")

Ready...


In [4]:
input = @cudaarray rand(28, 28, 1, 10)
w1 = ((rand(3,3,1,6).-0.5)./100)
w2 = ((rand(3,3,6,16).-0.5)./100)
println("Ready...")

Ready...


In [5]:
conv1 = convolve(input, w1, (2,2), (1,1))
x = conv1(input)
# pool1 = avgpooling(x, (2,2), (2,2), (1,1))
# x = pool1(x)
conv2 = convolve(x, w2, (2,2), (1,1))
x = conv2(x)
# pool2 = avgpooling(x, (2,2), (2,2), (1,1))
# x = pool2(x)
dense1 = dense(x, 120)
x = dense1(x)
dense2 = dense(x, 84)
x = dense2(x)
dense3 = dense(x, 10)
layers=[dense3,dense2,dense1,conv2,conv1]
println("Loaded")

Loaded


In [6]:
function f(x)
    x = conv1(x)
    x = relu(x)
#     x = pool1(x)
    
    x = conv2(x)
    x = relu(x)
#     x = pool2(x)
    
    x = dense1(x)
    x = relu(x)
    
    x = dense2(x)
    x = relu(x)
    
    x = dense3(x)
    x = softmax(x)
end

f (generic function with 1 method)

In [7]:
acc_train_plot = []
acc_test_plot = []
loss_train_plot = []
loss_test_plot = []

0-element Array{Any,1}

In [8]:
indices = sample(1:size(train)[1], size(train)[1], replace = false)

42000-element Array{Int64,1}:
 37083
 14904
 16110
 24051
  9584
 17329
 26068
 13981
 25644
 28926
 21502
  3572
 21132
     ⋮
  8260
 26084
 15633
 24762
  4823
 29768
  2862
 12608
 18598
 25928
   863
 32711

In [9]:
sample_batch_x, sample_batch_y = cat(xtrain[indices[(1):(100)]]..., dims=4), ytrain[indices[(1):(100)]]

([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

...

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Any[1, 10, 2, 4, 9, 6, 7, 2, 10, 7  …  2, 8, 7, 1, 10, 3, 3, 1, 1, 5])

In [11]:
dff(x,y) = @diff nll(f(x), y)
dv = dff(cu(sample_batch_x), sample_batch_y)

ArgumentError: ArgumentError: sum_outgrads(0.01,AutoGrad.UngetIndex(Float32[0.0983661 0.0983979 0.0983874 0.0983491 0.0983715 0.0983475 0.0983833 0.0983977 0.0983473 0.098347 0.0983656 0.0983944 0.0983575 0.0983762 0.0984243 0.0983807 0.0984258 0.0983656 0.0983827 0.0983874 0.0983815 0.0983751 0.0983801 0.0983848 0.0983594 0.098374 0.0984289 0.0983939 0.0983448 0.0983655 0.0983515 0.0984113 0.0983648 0.09838 0.0983728 0.0983864 0.0983717 0.0983535 0.0984308 0.0983654 0.0983773 0.0983597 0.098371 0.0983739 0.0983711 0.0983786 0.0983625 0.0983799 0.098369 0.0984209 0.0983847 0.0983833 0.0983914 0.098398 0.0983932 0.0983697 0.0983684 0.0984043 0.0983694 0.0983547 0.098386 0.0984138 0.0983852 0.0983692 0.0984268 0.0983745 0.0984304 0.0984054 0.0983891 0.0984453 0.0983991 0.0983557 0.0984117 0.0983996 0.0983486 0.0983659 0.0983795 0.098386 0.0983843 0.0983655 0.0983529 0.098389 0.0983875 0.0983922 0.0984309 0.0984362 0.098424 0.0983297 0.098432 0.0984009 0.0983849 0.0983911 0.0983347 0.0983

In [ ]:
dff(x,y) = @diff nll(f(x), y)

for e in 1:10
    println("Epoch ", e)
    indices = sample(1:size(train)[1], size(train)[1], replace = false)
    for (ei,si) in enumerate(1:batch:size(indices)[1])
        sample_batch_x, sample_batch_y = cat(xtrain[indices[(si):(ei*batch)]]..., dims=4), ytrain[indices[(si):(ei*batch)]]

        dv = dff(sample_batch_x, sample_batch_y)

        dparams = @parameters dv

        for pidx in 1:length(dparams[1:end-1])
            dw = grad(dv, dparams[pidx])
            layers[pidx].matrix .-=0.1*dw
        end
    end
    otrain = f(xtrain)
#     otest = f(xtest)
    push!(acc_train_plot, acc(otrain, ytrain))
    push!(loss_train_plot, nll(otrain, ytrain))
#     push!(acc_test_plot, acc(o, ytest))
#     push!(loss_test_plot, nll(o, ytest))
end

In [ ]:
plot([acc_plot], linewidth=2, title="Accuracy")

In [ ]:
plot([loss_plot], linewidth=2, title="Loss")